In [1]:
parent_folder = '/Users/username/Desktop/ML/Recommendations/arcane/'
from hydra import compose, initialize
import os

import xml.etree.ElementTree as ET

tree = ET.parse('../../../conf/application.run.xml')
root = tree.getroot()

envs_element = root.find('./configuration/envs')
for variable in envs_element.findall('env'):
    name = variable.get('name')
    value = variable.get('value')
    os.environ[name] = value

import sys
sys.path.append('/Users/username/Desktop/ML/Recommendations/arcane/')

from src._utils import load_bertopic_model_from_hf

In [2]:
import json

In [3]:
import numpy as np
import pandas as pd

### Getting pending articles

In [138]:
from sql.PostgresDatabaseOperation import PostgresDatabaseOperation

with PostgresDatabaseOperation() as cursor:
    sql = 'SELECT DISTINCT article_id FROM embeddings'
    cursor.execute(sql)
    results = cursor.fetchall()

all_article_ids = [x[0] for x in results]

In [291]:
# valid_articles = {}

# for k in insight_articles:
#     if 'is_premium_article' in k and k['is_premium_article'] == False:
#         article_id = k['_id']['$oid']
#         valid_articles[article_id] = k
#         valid_articles[article_id]['source_id'] = k['source_id']['$oid']

# valid_article_ids = list(valid_articles.keys())

In [292]:
# created_articles_df = pd.read_csv('all_article_attributes_WatermelonSapphireZipline.csv')

# created_articles = list(created_articles_df.article_id.unique())

In [139]:
from sql.PostgresDatabaseOperation import PostgresDatabaseOperation

In [140]:
with PostgresDatabaseOperation() as cursor:
    sql = 'SELECT DISTINCT article_id FROM llm_article_attributes'
    cursor.execute(sql)
    results = cursor.fetchall()

In [141]:
db_saved_results = [x[0] for x in results]

In [142]:
len(db_saved_results)

153957

In [143]:
pending_articles = list(set(all_article_ids) - set(db_saved_results))

In [144]:
len(pending_articles)

11

In [146]:
db_saved_results[20]

'651dc3b6a662d76276b7afd8'

In [145]:
pending_articles

['65316f1f1e5cc42b1b13f01b',
 '65316eaf1e5cc42b1b13e625',
 '65316eaf1e5cc42b1b13e623',
 '65316f121e5cc42b1b13ee36',
 '65316ea41e5cc42b1b13e5e2',
 '65316f151e5cc42b1b13eea8',
 '65316f1d1e5cc42b1b13efcc',
 '65316ebe1e5cc42b1b13e69e',
 '65316e9b1e5cc42b1b13e5b7',
 '65316f0a1e5cc42b1b13ed29',
 '65316ea21e5cc42b1b13e5da']

### Fetching function

In [2]:
from concurrent.futures import ThreadPoolExecutor, as_completed
import requests
import time
from datetime import datetime

In [3]:
def get_attributes_from_llm(article_id):
    url = 'http://Arcane-env.eba-mrsaixmg.ap-south-1.elasticbeanstalk.com/get_article_attributes_from_llm'
    r = requests.post(url, json = {'articleId': article_id})
    return article_id, r.text

In [125]:
def get_attributes_for_article_ids(article_ids):
    start_time = time.time()
    valid_attributes = {}
    completed = 0
    with ThreadPoolExecutor(max_workers=40) as executor:
        futures = [executor.submit(get_attributes_from_llm, art_id) for art_id in article_ids]
        
        for future in as_completed(futures):
            if future.result():
                art_id, response = future.result()
                if response:
                    try:
                        valid_attributes[art_id] = json.loads(response)
                        completed += 1
                    except:
                        pass
    print(f'Processed {completed} IDs in {time.time() - start_time} seconds at {datetime.now()}')
    return valid_attributes

In [15]:
a, b = get_attributes_from_llm('6555cab94b13023f93495fa6')

In [9]:
import json

In [16]:
json.loads(b)

{'analysis_of_article_sentiment': 'The sentiment in the article is neutral as it reports on facts about the current situation.',
 'analysis_of_article_type': 'The article presents factual information about a report on retirement savings of both sexes by Bank of America, with some inferences about the pay gaps.',
 'analysis_of_article_validity_duration': 'The report focuses on the findings of a 2022 report. Given its nature, the implications of the data would last for a week.',
 'analysis_of_financial_or_business_news': 'The article discusses retirement savings statistics and how they vary between men and women.',
 'analysis_of_popularity': 'This article might garner moderate interest as it talks about saving patterns, a common concern among the public.',
 'analysis_of_relevant_for_india': 'The news revolves around a US Banking report about retirement savings. While Indians might be interested in similar issues in the US, the specifics of the study would be less relevant.',
 'article_se

### Firing in chunks

In [135]:
chunk_size = 200
num_chunks = (len(pending_articles)//chunk_size) + 1
num_chunks

1

In [136]:
for i in range(num_chunks):
    cur_article_ids = pending_articles[i * chunk_size: (i+1)*chunk_size]
    get_attributes_for_article_ids(cur_article_ids)

Processed 0 IDs in 3.1425819396972656 seconds at 2023-12-10 13:06:15.110960


In [22]:
sample_articles = list(valid_articles.keys())[:10]

In [263]:
random_articles = np.random.choice(list(valid_articles.keys()), 20)

In [261]:
random_articles[0]

'651e1c02a662d76276b88e46'

In [212]:
get_attributes_from_llm(article_id=random_articles[0])

('65316ff01e5cc42b1b142a01',
 '{"analysis_of_article_sentiment": "The sentiment of the article is neutral as it provides factual details without expressing a specific positive or negative opinion.", "analysis_of_article_type": "The article is factual as it presents daily updates on Tata Steel stock prices.", "analysis_of_article_validity_duration": "This is a daily stock update which is not tied to any policy change or upcoming event, and hence, is not valid for long term.", "analysis_of_financial_or_business_news": "The article discusses the stock performance of Tata Steel, thus it falls under the financial news.", "analysis_of_popularity": "This information targets a specific audience i.e., stock traders or investors interested in Tata Steel, making it a niche piece.", "analysis_of_relevant_for_india": "Tata Steel is a key Indian corporation, hence its stock performances can be of interest to Indian investors and financial news enthusiasts.", "article_sentiment": "na", "article_type"

In [146]:
va = get_attributes_for_article_ids(random_articles)

Processed 15 IDs in 105.27986407279968 seconds at 2023-12-04 19:01:01.400113


In [135]:
111/23

4.826086956521739

In [76]:
def get_attributes_for_list_from_llm(article_ids):
    url = 'http://Arcane-env.eba-mrsaixmg.ap-south-1.elasticbeanstalk.com/get_article_attributes_from_llm'
    r = requests.post(url, json = {'articleIds': ','.join(article_ids)})
    return article_ids, r.text

In [264]:
random_articles[1]

'655c1d6e4b13023f934af568'

In [265]:
import time

In [266]:
start_time = time.time()
ax, bx = get_attributes_from_llm(random_articles[1])
print(f'done in {start_time - time.time()} seconds')

done in -31.320125818252563 seconds


In [267]:
bx

'{"analysis_of_article_sentiment": "The overall sentiment of the article is neutral, presenting various products and deals without favoring any particular company.", "analysis_of_article_type": "This article is fact-driven, providing a list of deals and products available ahead of Black Friday.", "analysis_of_article_validity_duration": "The validity of these deals is limited due to the impending Black Friday event, valid through November 24, 2023.", "analysis_of_financial_or_business_news": "This article focuses on early Black Friday deals, discussing a variety of products ranging from electronics to home goods and appliances.", "analysis_of_popularity": "The article targets consumers interested in Black Friday deals, making it niche. However, due to the widespread popularity of Black Friday, the content should garner a relatively wide audience.", "analysis_of_relevant_for_india": "Despite being based in the US, the article pertains to global consumer behavior, particularly surroundin

In [111]:
url = 'http://Arcane-env.eba-mrsaixmg.ap-south-1.elasticbeanstalk.com/get_article_attributes_from_llm'

In [120]:
r = requests.post(url, json = {'articleId': '6555cc3f4b13023f934a6c2b'})

In [121]:
r.text

'{"analysis_of_article_sentiment": "The sentiment of the article is mixed, with negative sentiments regarding the hike in prices and positive sentiments about the impact on the EV industry.", "analysis_of_article_type": "The article is largely factual, covering specific developments and announcements from Ather Energy about price hikes, regulation compliance, and the impact on the EV industry.", "analysis_of_article_validity_duration": "The article covers a short-term price hike, future plans of the company, and regulatory compliance. It has validity over a short period.", "analysis_of_financial_or_business_news": "The article talks about a price hike by Ather Energy for its electric scooter, its reasoning behind it, the consequences of the price hike and its impacts on its products, production plans and future plans.", "analysis_of_popularity": "The article would appeal to a niche audience including those interested in the EV industry, economic policies, price hikes, business strategi

In [117]:
import uuid

In [77]:
random_articles[0]

'6555cc3f4b13023f934a6c2b'

In [109]:
random_articles[0]

'651dee54a662d76276b820fb'

In [106]:
a, b = get_attributes_from_llm(random_articles[0])

In [107]:
b

'<!doctype html>\n<html lang=en>\n<title>500 Internal Server Error</title>\n<h1>Internal Server Error</h1>\n<p>The server encountered an internal error and was unable to complete your request. Either the server is overloaded or there is an error in the application.</p>\n'

In [92]:
json.loads(b).keys()

dict_keys(['651dcf62a662d76276b7cf37', '651e0ce0a662d76276b86b12', '653168de1e5cc42b1b13bf01', '65316f5d1e5cc42b1b13fbb5', '6555c3ab4b13023f9348d52e', '6555c8144b13023f934905d3', '6555ca924b13023f934947f2', '6555caac4b13023f93495712', '6555cc3f4b13023f934a6c2b', '655c1eb74b13023f934af585'])

In [96]:
json.loads(b)['653168de1e5cc42b1b13bf01']

{'analysis_of_article_sentiment': 'The sentiment in the article is bullish since it discusses a growth in leasing',
 'analysis_of_article_type': 'The article is factual, presenting information on a research report and giving data on rents',
 'analysis_of_article_validity_duration': 'The article has a specific timeframe i.e., the first half of 2023. Its validity would be a couple of months until a new report is released',
 'analysis_of_financial_or_business_news': 'The article covers data on retail leasing in Delhi-NCR, how it has grown, and the trend in rents',
 'analysis_of_popularity': 'The article would be of moderate interest as it covers a growth in a specific geographical area and discusses popular brands',
 'analysis_of_relevant_for_india': 'The information is specifically relevant for India, talking about changes in property trends in Delhi-NCR',
 'article_sentiment': 'bull',
 'article_type': 'fact',
 'article_validity_duration': 30,
 'final_summary': 'In H1 2023 Delhi-NCR saw 

In [80]:
a

'6555cc3f4b13023f934a6c2b'

In [79]:
b

'{}\n'

In [75]:
','.join(random_articles)

'6555cc3f4b13023f934a6c2b,651dcf62a662d76276b7cf37,651e0ce0a662d76276b86b12,65316f5d1e5cc42b1b13fbb5,6555c3ab4b13023f9348d52e,6555caac4b13023f93495712,6555c8144b13023f934905d3,655c1eb74b13023f934af585,653168de1e5cc42b1b13bf01,6555ca924b13023f934947f2'

In [64]:
random_articles

array(['6555cc3f4b13023f934a6c2b', '651dcf62a662d76276b7cf37',
       '651e0ce0a662d76276b86b12', '65316f5d1e5cc42b1b13fbb5',
       '6555c3ab4b13023f9348d52e', '6555caac4b13023f93495712',
       '6555c8144b13023f934905d3', '655c1eb74b13023f934af585',
       '653168de1e5cc42b1b13bf01', '6555ca924b13023f934947f2'],
      dtype='<U24')

In [71]:
a, b = get_attributes_for_list_from_llm(random_articles)

In [73]:
a

array(['6555cc3f4b13023f934a6c2b', '651dcf62a662d76276b7cf37',
       '651e0ce0a662d76276b86b12', '65316f5d1e5cc42b1b13fbb5',
       '6555c3ab4b13023f9348d52e', '6555caac4b13023f93495712',
       '6555c8144b13023f934905d3', '655c1eb74b13023f934af585',
       '653168de1e5cc42b1b13bf01', '6555ca924b13023f934947f2'],
      dtype='<U24')

In [74]:
b

'<html>\r\n<head><title>504 Gateway Time-out</title></head>\r\n<body>\r\n<center><h1>504 Gateway Time-out</h1></center>\r\n<hr><center>nginx</center>\r\n</body>\r\n</html>\r\n'

In [60]:
len(b.keys())

8

In [61]:
a.keys()

dict_keys(['6555cbf54b13023f934a364f', '651e1916a662d76276b887ab'])

In [62]:
b

{'65316f701e5cc42b1b140028': '<html>\r\n<head><title>504 Gateway Time-out</title></head>\r\n<body>\r\n<center><h1>504 Gateway Time-out</h1></center>\r\n<hr><center>nginx</center>\r\n</body>\r\n</html>\r\n',
 '6555c0934b13023f9348cbe5': '<html>\r\n<head><title>504 Gateway Time-out</title></head>\r\n<body>\r\n<center><h1>504 Gateway Time-out</h1></center>\r\n<hr><center>nginx</center>\r\n</body>\r\n</html>\r\n',
 '6555c7ea4b13023f9348edbb': '<html>\r\n<head><title>504 Gateway Time-out</title></head>\r\n<body>\r\n<center><h1>504 Gateway Time-out</h1></center>\r\n<hr><center>nginx</center>\r\n</body>\r\n</html>\r\n',
 '651dd820a662d76276b7e662': '<html>\r\n<head><title>504 Gateway Time-out</title></head>\r\n<body>\r\n<center><h1>504 Gateway Time-out</h1></center>\r\n<hr><center>nginx</center>\r\n</body>\r\n</html>\r\n',
 '651dc9c2a662d76276b7bfd3': '<html>\r\n<head><title>504 Gateway Time-out</title></head>\r\n<body>\r\n<center><h1>504 Gateway Time-out</h1></center>\r\n<hr><center>nginx</c

In [56]:
b

{'651e1d97a662d76276b89226': '<html>\r\n<head><title>504 Gateway Time-out</title></head>\r\n<body>\r\n<center><h1>504 Gateway Time-out</h1></center>\r\n<hr><center>nginx</center>\r\n</body>\r\n</html>\r\n',
 '6555bfb74b13023f9348c81c': '<html>\r\n<head><title>504 Gateway Time-out</title></head>\r\n<body>\r\n<center><h1>504 Gateway Time-out</h1></center>\r\n<hr><center>nginx</center>\r\n</body>\r\n</html>\r\n',
 '65316f341e5cc42b1b13f39b': '<html>\r\n<head><title>504 Gateway Time-out</title></head>\r\n<body>\r\n<center><h1>504 Gateway Time-out</h1></center>\r\n<hr><center>nginx</center>\r\n</body>\r\n</html>\r\n',
 '6555cc174b13023f934a4e3e': '<html>\r\n<head><title>504 Gateway Time-out</title></head>\r\n<body>\r\n<center><h1>504 Gateway Time-out</h1></center>\r\n<hr><center>nginx</center>\r\n</body>\r\n</html>\r\n',
 '651e03bfa662d76276b8558e': '<html>\r\n<head><title>504 Gateway Time-out</title></head>\r\n<body>\r\n<center><h1>504 Gateway Time-out</h1></center>\r\n<hr><center>nginx</c

In [38]:
sample_articles[:2]

['6531690a1e5cc42b1b13cb4e', '653168f41e5cc42b1b13c4fa']

In [36]:
json.loads(b['653168f41e5cc42b1b13c4fa'])

{'analysis_of_article_sentiment': 'The sentiment is bearish due to the negative outcome in the court case to Zee Entertainment.',
 'analysis_of_article_type': 'This article is factual, presenting the particular event of ZEE Entertainment giving a detailed account of the situation.',
 'analysis_of_article_validity_duration': 'This news news of business and legal matters having financial implications and information that may change, it should be valid for a few days.',
 'analysis_of_financial_or_business_news': 'The article pertains to business, speaking about the current legal dispute between two media networks about the proposed merger.',
 'analysis_of_popularity': "It's a business news about two well-known media enterprises - Zee and Sony. It has potential for moderate popularity.",
 'analysis_of_relevant_for_india': 'The event focuses on a business merger operation encompassing Indian and foreign entities. It holds relevance for Indian readers.',
 'article_sentiment': 'bear',
 'artic

In [37]:
b

{'653168f41e5cc42b1b13c4fa': '{"analysis_of_article_sentiment":"The sentiment is bearish due to the negative outcome in the court case to Zee Entertainment.","analysis_of_article_type":"This article is factual, presenting the particular event of ZEE Entertainment giving a detailed account of the situation.","analysis_of_article_validity_duration":"This news news of business and legal matters having financial implications and information that may change, it should be valid for a few days.","analysis_of_financial_or_business_news":"The article pertains to business, speaking about the current legal dispute between two media networks about the proposed merger.","analysis_of_popularity":"It\'s a business news about two well-known media enterprises - Zee and Sony. It has potential for moderate popularity.","analysis_of_relevant_for_india":"The event focuses on a business merger operation encompassing Indian and foreign entities. It holds relevance for Indian readers.","article_sentiment":"be